In [1]:
from sklearn.datasets import load_digits
from superintendent import ClassLabeller
import numpy as np
from skimage.io import imread
import os
from pathlib import Path
import numpy as np
import glob
import IPython

In [2]:
data_dir = "/home/jovyan/data/chips"
subdirs = os.listdir(data_dir)

In [33]:
img_ndwis = glob.glob(os.path.join(data_dir, "lat_40_15_lon_26_39", "img_08*.png"))

In [34]:
len(img_ndwis)

30

In [35]:
from skimage.io import imread
data_to_label = np.stack([imread(img) for img in img_ndwis])

In [36]:
data_to_label.shape

(30, 100, 100)

In [37]:
import IPython.display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from PIL import Image
def cmap_display_function(feature):
    IPython.display.display(widgets.Box(layout=widgets.Layout(height="2.5%")))
    feature = -feature
    cm = plt.get_cmap('RdYlBu')
    colored_image = cm(feature)
    feature = Image.fromarray(np.uint8(colored_image * 255))
    IPython.display.display(feature)
    IPython.display.display(widgets.Box(layout=widgets.Layout(height="2.5%")))

In [38]:
widget = ClassLabeller.from_images(
    features=data_to_label,
    canvas_size=(200, 200),
    options=range(10),
    display_func=cmap_display_function
)

In [39]:
widget

ClassLabeller(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.0),)), Box(chil…

In [39]:
len(widget.new_labels)

64

In [40]:
len(img_ndwis)

64

In [27]:
import pandas as pd
df = pd.DataFrame(columns=['path', 'count'])

In [28]:
df["path"] = img_ndwis

In [30]:
df["count"] = widget.new_labels

In [31]:
df

,path,count
0,/home/jovyan/data/chips/lat_58_14_lon_7_99/img...,0
1,/home/jovyan/data/chips/lat_58_14_lon_7_99/img...,0
2,/home/jovyan/data/chips/lat_44_1_lon_28_71/img...,3
3,/home/jovyan/data/chips/lat_44_1_lon_28_71/img...,3
4,/home/jovyan/data/chips/lat_44_1_lon_28_71/img...,0
...,...,...
525,/home/jovyan/data/chips/lat_57_07_lon_24_01/im...,5
526,/home/jovyan/data/chips/lat_57_07_lon_24_01/im...,4
527,/home/jovyan/data/chips/lat_57_07_lon_24_01/im...,2
528,/home/jovyan/data/chips/lat_57_07_lon_24_01/im...,3
